# Testing model 
Here we will test and change the model to understand the features so we can change/modify the model

In [1]:
from os.path import join
import os

ROOT = '/home/ashar'     # default for the drive
CODE = 'LaneNet'       # path to our code on Drive
DATASET = 'dataset/trainset'
EXPERIMENT = 'LaneNet/config'
MODEL_SAVE = 'pretrained'
PRETRAINED = 'pretrained'

CODE_PATH = join(ROOT, CODE)
DATASET_PATH = join(ROOT, DATASET)
EXPERIMENT_CONFIG = join(ROOT, EXPERIMENT)
MODEL_SAVE = join(ROOT, MODEL_SAVE)
PRETRAINED_PATH = join(ROOT, PRETRAINED)

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('{}'.format(CODE_PATH))

### Performing some imports
Performing imports necessary for us to be able to import all the features


In [3]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import json
import os
import shutil
import time

import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm



#from config import *
import data
from utils.transforms import *
from utils.lr_scheduler import PolyLR

# Location of our configuration for training
exp_dir = EXPERIMENT_CONFIG + '/cfg.json'
exp_name = 'training_accuracy_test'
save_dir = MODEL_SAVE + '/' + exp_name
with open(exp_dir) as f:
  exp_cfg = json.load(f)

# Extracting important configurations
resize_shape = tuple(exp_cfg['dataset']['resize_shape'])

device = torch.device(exp_cfg['device'])


### Loading data for testing model


In [5]:
# Training data
# Predefined mean and std- deviation
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

transform_train = Compose(Resize(resize_shape), Rotation(2), ToTensor(),
                          Normalize(mean=mean, std=std))

import data

dataset_name = exp_cfg['dataset']['dataset_name']
Dataset_Type = getattr(data, dataset_name)

# Training Data
train_dataset = Dataset_Type(DATASET_PATH, "train", transform_train)
train_loader = DataLoader(train_dataset, batch_size=exp_cfg['dataset']['batch_size'], shuffle=True, collate_fn=train_dataset.collate, num_workers=8)

# Validation Data
transform_val_img = Resize(resize_shape)
transform_val_x = Compose(ToTensor(), Normalize(mean=mean, std=std))
transform_val = Compose(transform_val_img, transform_val_x)
val_dataset = Dataset_Type(DATASET_PATH, "val", transform_val)
#val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=val_dataset.collate, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=20, collate_fn=val_dataset.collate, num_workers=4)

### Determining accuracy and initializing model

In [6]:




##################################################################
########
#########
##################################################################

# Extra json files containing dictionary infos
train_accuracy_path = os.path.join(DATASET_PATH, "train_accuracy.json")
val_accuracy_path = os.path.join(DATASET_PATH, "val_accuracy.json")
test_accuracy_path = os.path.join(DATASET_PATH, "test_accuracy.json")

with open(train_accuracy_path) as f:
  train_accuracy_dict = json.load(f)

with open(val_accuracy_path) as f:
  val_accuracy_dict = json.load(f)  

with open(test_accuracy_path) as f:
  test_accuracy_dict = json.load(f)  

from utils.prob2lines import getLane
from utils.lane_evaluation.tusimple.lane import LaneEval
import torch.nn.functional as F

# This generates the lane dictionary we need
def generate_lanes(seg_label, exist_label, num_pts):
  # For prediction
  seg = seg_label
  exist = [1 if exist_label[i] > 0.5 else 0 for i in range(4)]
  lane_coords = getLane.prob2lines_tusimple(seg, exist, resize_shape=(720, 1280), y_px_gap=10, pts = num_pts)
  for i in range(len(lane_coords)):
      lane_coords[i] = sorted(lane_coords[i], key=lambda pair: pair[1])
  lanes = []  # Empty list representing lanes
  h_samples = [] # Empty list representing h_samples
  # Now let's make a dictionary
  for l in lane_coords:
      if len(l) == 0:
          continue
      lanes.append([])
      for (x, y) in l:
          lanes[-1].append(int(x))
  #for (x, y) in lane_coords[0]:
      #h_samples.append(y)
  return lanes


def train_accuracy(gt_lanes, y_samples, seg_pred, exist_pred):
  # Workaround 
  # First we will make the lanes for each of the labels
  pred_lanes = generate_lanes(seg_pred, exist_pred, len(y_samples))
  
  accuracy, positives, negatives = LaneEval.bench(pred_lanes, gt_lanes, y_samples, 0)
  # Lets call the function eval
  return accuracy

## Train step for testing our model

In [7]:
# Inserting the whole model here for prototyping
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


class SCNN(nn.Module):
    def __init__(
            self,
            input_size,
            ms_ks=9,
            pretrained=True
    ):
        """
        Argument
            ms_ks: kernel size in message passing conv
        """
        super(SCNN, self).__init__()
        print("no")
        self.pretrained = pretrained
        self.net_init(input_size, ms_ks)
        if not pretrained:
            self.weight_init()

        self.scale_background = 0.4
        self.scale_seg = 1.0
        self.scale_exist = 0.1

        self.ce_loss = nn.CrossEntropyLoss(weight=torch.tensor([self.scale_background, 1, 1, 1, 1]))
        self.bce_loss = nn.BCELoss()

    def forward(self, img, seg_gt=None, exist_gt=None):
        x = self.backbone(img)
        print(x.shape)
        x = self.layer1(x)
        x = self.message_passing_forward(x)
        x = self.layer2(x)

        seg_pred = F.interpolate(x, scale_factor=8, mode='bilinear', align_corners=True)
        x = self.layer3(x)
        print(x.shape)
        x = x.view(-1, self.fc_input_feature)
        print(x.shape)
        exist_pred = self.fc(x)

        if seg_gt is not None and exist_gt is not None:
            loss_seg = self.ce_loss(seg_pred, seg_gt)
            loss_exist = self.bce_loss(exist_pred, exist_gt)
            loss = loss_seg * self.scale_seg + loss_exist * self.scale_exist
        else:
            loss_seg = torch.tensor(0, dtype=img.dtype, device=img.device)
            loss_exist = torch.tensor(0, dtype=img.dtype, device=img.device)
            loss = torch.tensor(0, dtype=img.dtype, device=img.device)

        return seg_pred, exist_pred, loss_seg, loss_exist, loss

    def message_passing_forward(self, x):
        Vertical = [True, True, False, False]
        Reverse = [False, True, False, True]
        for ms_conv, v, r in zip(self.message_passing, Vertical, Reverse):
            x = self.message_passing_once(x, ms_conv, v, r)
        return x

    def message_passing_once(self, x, conv, vertical=True, reverse=False):
        """
        Argument:
        ----------
        x: input tensor
        vertical: vertical message passing or horizontal
        reverse: False for up-down or left-right, True for down-up or right-left
        """
        nB, C, H, W = x.shape
        if vertical:
            slices = [x[:, :, i:(i + 1), :] for i in range(H)]
            dim = 2
        else:
            slices = [x[:, :, :, i:(i + 1)] for i in range(W)]
            dim = 3
        if reverse:
            slices = slices[::-1]

        out = [slices[0]]
        for i in range(1, len(slices)):
            out.append(slices[i] + F.relu(conv(out[i - 1])))
        if reverse:
            out = out[::-1]
        return torch.cat(out, dim=dim)

    def net_init(self, input_size, ms_ks):
        input_w, input_h = input_size
        #self.fc_input_feature = 5 * int(input_w/16) * int(input_h/16)
        self.fc_input_feature = 5 * int(input_w/16) * int(input_h/16)
        """
        self.backbone = models.vgg16_bn(pretrained=self.pretrained).features
        print(self.backbone)

        # ----------------- process backbone -----------------
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
        #summary(self.backbone().to(device), (3, 224, 224))
        
        
        for i in [34, 37, 40]:
            conv = self.backbone._modules[str(i)]
            dilated_conv = nn.Conv2d(
                conv.in_channels, conv.out_channels, conv.kernel_size, stride=conv.stride,
                padding=tuple(p * 2 for p in conv.padding), dilation=2, bias=(conv.bias is not None)
            )
            dilated_conv.load_state_dict(conv.state_dict())
            self.backbone._modules[str(i)] = dilated_conv
        self.backbone._modules.pop('33')
        self.backbone._modules.pop('43')
        """
        
        ################################## RESNET 18 BACKBONE########################################
        model = models.resnet18(pretrained=True)
        ## Extracting the model layers as elementst of a list
        mod = list(model.children())
        # Removing all layers after layer 33
        #for i in range(33):
        mod.pop()
        mod.pop()
        mod.pop()
        mod.pop()
        convolutional = nn.Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False)
        relu = nn.ReLU(inplace = True)
        model = torch.nn.Sequential(*mod, convolutional, relu)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
        self.backbone = model.to(device)
        ###############################################################################################
        
        #print(self.backbone)
        
        # ----------------- SCNN part -----------------
        self.layer1 = nn.Sequential(
            nn.Conv2d(512, 1024, 3, padding=4, dilation=4, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Conv2d(1024, 128, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU()  # (nB, 128, 36, 100)
        )

        # ----------------- add message passing -----------------
        self.message_passing = nn.ModuleList()
        self.message_passing.add_module('up_down', nn.Conv2d(128, 128, (1, ms_ks), padding=(0, ms_ks // 2), bias=False))
        self.message_passing.add_module('down_up', nn.Conv2d(128, 128, (1, ms_ks), padding=(0, ms_ks // 2), bias=False))
        self.message_passing.add_module('left_right',
                                        nn.Conv2d(128, 128, (ms_ks, 1), padding=(ms_ks // 2, 0), bias=False))
        self.message_passing.add_module('right_left',
                                        nn.Conv2d(128, 128, (ms_ks, 1), padding=(ms_ks // 2, 0), bias=False))
        # (nB, 128, 36, 100)

        # ----------------- SCNN part -----------------
        self.layer2 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.Dropout2d(0.5),
            nn.Conv2d(128, 5, 1)  # get (nB, 5, 36, 100)
        )

        self.layer3 = nn.Sequential(
            nn.Softmax(dim=1),  # (nB, 5, 36, 100)
            nn.AvgPool2d(2, 2),  # (nB, 5, 18, 50)
        )
        self.fc = nn.Sequential(
            nn.Linear(self.fc_input_feature, 128),
            nn.ReLU(),
            nn.Linear(128, 4),
            nn.Sigmoid()
        )

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.reset_parameters()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data[:] = 1.
                m.bias.data.zero_()


In [9]:
net = SCNN(resize_shape, pretrained=True)
net = net.to(device)
net = torch.nn.DataParallel(net)

optimizer = optim.SGD(net.parameters(), **exp_cfg['optim'])
lr_scheduler = PolyLR(optimizer, 0.9, **exp_cfg['lr_scheduler'])
best_val_loss = 1e6

# Release CUDA memory
torch.cuda.empty_cache()

training_accuracy = 0 # Accuracy per epoch
net.train()

h_sample = str(-1)  # For accuracy
for sample in train_loader:
    train_loss_batch = 0  # Training loss for each batch
    img = sample['img'].to(device)
    segLabel = sample['segLabel'].to(device)
    exist = sample['exist'].to(device)
    
    optimizer.zero_grad()
    seg_pred, exist_pred, loss_seg, loss_exist, loss = net(img, segLabel, exist)
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
    # Detaching for out of memory errors
    seg_pred_to_pass = F.softmax(seg_pred, dim=1)
    seg_pred_to_pass = seg_pred_to_pass.detach().cpu().numpy()
    seg_pred = seg_pred.detach().cpu().numpy()
    exist_pred = exist_pred.detach().cpu().numpy()

    # Accuracy part
    for b in range(len(seg_pred)):
        lane_index = b
        accuracy = train_accuracy(train_accuracy_dict[lane_index][str(lane_index)], train_accuracy_dict[lane_index][h_sample], seg_pred_to_pass[b], exist_pred[b])
        training_accuracy += accuracy
        #print(accuracy)
    
    if isinstance(net, torch.nn.DataParallel):
            loss_seg = loss_seg.sum()
            loss_exist = loss_exist.sum()
            loss = loss.sum()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    train_loss = loss.item()
    train_loss_seg = loss_seg.item()
    train_loss_exist = loss_exist.item()

    lr = optimizer.param_groups[0]['lr']
        
    break

no
torch.Size([32, 512, 36, 64])
torch.Size([32, 5, 18, 32])
torch.Size([32, 2880])
